# Titanic: Machine Learning from Disaster

### Desciption:
#### The sinking of the RMS Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.
#### One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.
#### In this contest, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

### This Notebook will show basic examples of:
### Data Handling
#### - Importing Data with Pandas
#### - Cleaning Data
#### - Exploring Data through Visualizations with Matplotlib
### Data Analysis
#### - Logit Regression Model (Using Scikit learn)

In [7]:
# Necessarry packages 
#################################################
# Handling data frame
import pandas as pd
import numpy as np

## Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("paper")
sns.set(font_scale=1.25)

### Linear Regression Model
from sklearn.preprocessing import Imputer, LabelBinarizer, StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

###############################################################################
# Read, Cleaning & transforming Data
###############################################################################
# Read
training_data = pd.read_csv('../../data/train.csv')

# Cleaning
## Remove unnecessary colums
training_data.drop(["Name", "Ticket"], axis=1, inplace = True)

Pclass = np.sort(list(set(training_data["Pclass"])))
for i in range(len(Pclass)):
    training_data["Pclass_" + str(i+1)] = [ 1 if training_data["Pclass"][j]==Pclass[i] else 0 for j in range(training_data.shape[0])]

Sex = np.sort(list(set(training_data["Sex"])))
for i in range(len(Sex)):
    training_data["Sex_" + str(i+1)] = [ 1 if training_data["Sex"][j]==Sex[i] else 0 for j in range(training_data.shape[0])]

#Cabin = np.sort(list(set(training_data["Cabin"])))
#Cabin = Cabin[0:len(Cabin)-1]
#for i in range(len(Cabin)):
#    training_data["Cabin_" + str(i+1)] = [ 1 if training_data["Cabin"][j]==Cabin[i] else 0 for j in range(training_data.shape[0])]

Embarked = np.sort(list(set(training_data["Embarked"])))
Embarked = Embarked[0:len(Embarked)-1]
for i in range(len(Embarked)):
    training_data["Embarked_" + str(i+1)] = [ 1 if training_data["Embarked"][j]==Embarked[i] else 0 for j in range(training_data.shape[0])]

### handle NA value (missing value)
#####################################################################
## Strategy 1: Removing Nan                                        ##
#####################################################################

training_data_removed_nan = training_data[~np.isnan(training_data.Age)]

#####################################################################
## Strategy 2: Replacing Nan with Mean                             ##
#####################################################################

tmp = np.round(np.mean(training_data["Age"]))
training_data_replaced_nan_with_mean = training_data
for i in range(training_data_replaced_nan_with_mean.shape[0]):
    if np.isnan(training_data_replaced_nan_with_mean["Age"][i]) == True:
        training_data_replaced_nan_with_mean.set_value(i, "Age", tmp)
                                          
#####################################################################
## Strategy 3: Replacing Nan with Median                           ##
#####################################################################

tmp = np.round(np.median(training_data["Age"]))
training_data_replaced_nan_with_median = training_data
for i in range(training_data_replaced_nan_with_median.shape[0]):
    if np.isnan(training_data_replaced_nan_with_median["Age"][i]) == True:
        training_data_replaced_nan_with_median.set_value(i, "Age", tmp)
        
# You want to save data to file? 
#training_data.to_csv("../../data/cleaned_train_removed_nan_python.csv")
#training_data_replaced_nan_with_mean.to_csv("../../data/cleaned_train_replaced_nan_with_mean_python.csv")
#training_data_replaced_nan_with_median.to_csv("../../data/cleaned_train_replaced_nan_with_median_python.csv")

#####################################################################
# Data Visualization                                                #
#####################################################################
# Normal Matplotlib plot
plt.figure(figsize=(10, 20))
# plots a bar graph of those who surived vs those who did not. 
ax1 = plt.subplot2grid((5,2),(0,0))              
ax1 = training_data.Survived.value_counts().plot(kind='bar')
plt.title("Distribution of Survival, (1 = Survived)")    

ax2 = plt.subplot2grid((5,2),(0,1))
ax2 = plt.scatter(training_data.Survived, training_data.Age)
plt.ylabel("Age")                          
#plt.grid(b=True, which='major', axis='y')  
plt.title("Survival by Age,  (1 = Survived)")

# Plot from Seaborn package
ax3 = plt.subplot2grid((5,2), (1, 0))
ax3 = sns.distplot(training_data["Survived"], kde=False)
plt.title("Histogram")

ax4 = plt.subplot2grid((5,2), (1, 1))
ax4 = sns.boxplot(x=training_data["Survived"], y=training_data["Pclass"])
plt.title("Variation of Survived with Pclass")

ax5 = plt.subplot2grid((5,2), (2, 0))
ax5 = sns.boxplot(x=training_data["Survived"], y=training_data["Sex"])
plt.title("Variation of Survived with Sex")

ax6 = plt.subplot2grid((5,2), (2, 1))
ax6 = sns.boxplot(x=training_data["Survived"], y=training_data["Age"])
plt.title("Variation of Survived with Age")

ax7 = plt.subplot2grid((5,2), (3, 0))
ax7 = sns.boxplot(x=training_data["Survived"], y=training_data["SibSp"])
plt.title("Variation of Survived with SibSp")

ax8 = plt.subplot2grid((5,2), (3, 1))
ax8 = sns.boxplot(x=training_data["Survived"], y=training_data["Parch"])
plt.title("Variation of Survived with Parch")

ax9 = plt.subplot2grid((5,2), (4, 0), colspan=2)
ax9 = sns.boxplot(x=training_data["Survived"], y=training_data["Fare"])
plt.title("Variation of Survived with Fare")
plt.tight_layout()

#####################################################################
# Splitting Training Dataset into Training and Testing Set          #
#####################################################################

all_columns = training_data.columns
tabu_list = ["PassengerId", "Survived", "Pclass", "Sex", "Cabin", "Embarked"]
x_columns = []
for i in range(len(all_columns)):
    if all_columns[i] in tabu_list:
        continue
    else:
        x_columns.append(i)
x_columns = np.sort(x_columns)
#####################################################################
## Strategy 1: Removing Nan                                        ##
#####################################################################
x_values_removed_nan = training_data_removed_nan[x_columns].values
y_values_removed_nan = training_data_removed_nan["Survived"].values
x_values_removed_nan_training_set, x_values_removed_nan_validation_set, y_values_removed_nan_training_set, y_values_removed_nan_validation_set = train_test_split(x_values_removed_nan, y_values_removed_nan, test_size=.2)

#####################################################################
## Strategy 2: Replacing Nan with Mean                             ##
#####################################################################
x_values_replaced_nan_with_mean = training_data_replaced_nan_with_mean[x_columns].values
y_values_replaced_nan_with_mean = training_data_replaced_nan_with_mean["Survived"].values
x_values_replaced_nan_with_mean_training_set, x_values_replaced_nan_with_mean_validation_set, y_values_replaced_nan_with_mean_training_set, y_values_replaced_nan_with_mean_validation_set = train_test_split(x_values_replaced_nan_with_mean, y_values_replaced_nan_with_mean, test_size=.2)

#####################################################################
## Strategy 3: Replacing Nan with Median                           ##
#####################################################################
x_values_replaced_nan_with_median = training_data_replaced_nan_with_median[x_columns].values
y_values_replaced_nan_with_median = training_data_replaced_nan_with_median["Survived"].values
x_values_replaced_nan_with_median_training_set, x_values_replaced_nan_with_median_validation_set, y_values_replaced_nan_with_median_training_set, y_values_replaced_nan_with_median_validation_set = train_test_split(x_values_replaced_nan_with_median, y_values_replaced_nan_with_median, test_size=.2)

#####################################################################
# Models                                                            #
#####################################################################

#####################################################################
## Training with Scikit Learn                                      ##
#####################################################################

#####################################################################
### Logistic Regression                                           ###
#####################################################################

#####################################################################
#### Default Setting                                             ####
#####################################################################

lr_model = LogisticRegression(fit_intercept=True)
lr_model.fit(x_values_replaced_nan_with_mean_training_set, y_values_replaced_nan_with_mean_training_set)
lr_model_predictions = lr_model.predict(x_values_replaced_nan_with_mean_validation_set)
lr_confusion_matrix = confusion_matrix(y_values_replaced_nan_with_mean_validation_set, lr_model_predictions)

print(lr_model.coef_)
print("Accuracy of Logistic Regression = " +str((np.sum(lr_model_predictions == y_values_replaced_nan_with_mean_validation_set) / len(y_values_replaced_nan_with_mean_validation_set))*100))
print("Confusion Matrix of Logistic Regression = ")
print(lr_confusion_matrix)

In [ ]:
# Backup code for upgrading packages (copy and paste to Command Prompt to run)
conda install seaborn && conda update pyzmq numpy scipy sympy scikit-learn ipython numba matplotlib pandas blaze jupyter spyder